<a href="https://colab.research.google.com/github/ujwal5/Transcriber/blob/main/WhisperX_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Transcriber
  !pip install git+https://github.com/m-bain/whisperx.git
!pip install pyannote.audio

In [ ]:
import whisperx
import gc

In [ ]:
class Transcriber:
    def __init__(self):
        self.device = "cuda"  # Set the device for processing (e.g., "cuda" for GPU)
        self.batch_size = 16   # Set batch size for transcription (reduce if low on GPU memory)
        self.compute_type = "float16"   # Set compute type for optimization (change to "int8" if low on GPU memory, may reduce accuracy)
        self.model = whisperx.load_model("large-v2", self.device, compute_type=self.compute_type) # Load the WhisperX model for transcription
        self.diarize_model = whisperx.DiarizationPipeline(use_auth_token="ENTER YOUR TOKEN FROM HUGGING FACE ", device=self.device)  # Load the WhisperX diarization model

    def get_transcribe_dict(self):
        self.audio = whisperx.load_audio(self.audio_file)  # Load the audio file for transcription
        self.result = self.model.transcribe(self.audio, batch_size=self.batch_size)   # Perform transcription using the loaded model
        self.model_a, self.metadata = whisperx.load_align_model(language_code=self.result["language"], device=self.device) # Load alignment model for further processing
        self.result = whisperx.align(self.result["segments"], self.model_a, self.metadata, self.audio, self.device, return_char_alignments=False)  # Align the transcription segments
        self.diarize_segments = self.diarize_model(self.audio) # Perform speaker diarization on the audio
        self.result = whisperx.assign_word_speakers(self.diarize_segments, self.result)  # Assign speakers to transcription segments


        return self.result  # Return the transcribed result with speaker information






In [ ]:
transcriber = Transcriber() # Create an instance of the Transcriber class and assign it to the variable "transcriber"


In [ ]:
transcriber.audio_file = "/content/Untitled.mp4" # Set the path to the audio file for transcription
result = transcriber.get_transcribe_dict() # Perform transcription and speaker diarization, and store the result in the "result" variable
for item in result['segments']: # Iterate over each segment in the transcription result
    print(item.get('text')) # Print the text content of the segment
    print(item.get('speaker'))  # Print the speaker ID of the segment
